In [ ]:
# reset_dir.ipynb
# Full clean slate: drop bronze/silver/enriched tables and delete their storage and checkpoints.

from kflow.config import bronze_paths, silver_paths

# 1. Drop metastore Delta tables (with PURGE to remove lingering metadata)
to_drop = [
    "kardia_bronze.bronze_encounters",
    "kardia_bronze.bronze_claims",
    "kardia_bronze.bronze_patients",
    "kardia_bronze.bronze_providers",
    "kardia_bronze.bronze_feedback",
    "kardia_silver.silver_encounters",
    "kardia_silver.silver_claims",
    "kardia_silver.silver_patients",
    "kardia_silver.silver_providers",
    "kardia_silver.silver_feedback",
    "kardia_silver.silver_encounters_enriched",  # enriched target
]

for full_name in to_drop:
    print(f"Dropping table {full_name}")
    spark.sql(f"DROP TABLE IF EXISTS {full_name} PURGE")

# 2. Remove underlying storage and checkpoints
for name in ("encounters", "claims", "patients", "providers", "feedback"):
    # bronze layer
    P = bronze_paths(name)
    print(f"Removing bronze data and checkpoint for {name}: {P.bronze}, {P.checkpoint}")
    try:
        dbutils.fs.rm(P.bronze, recurse=True)
    except Exception as e:
        print(f"  (ignore) failed removing {P.bronze}: {e}")
    try:
        dbutils.fs.rm(P.checkpoint, recurse=True)
    except Exception as e:
        print(f"  (ignore) failed removing {P.checkpoint}: {e}")

    # silver layer
    S = silver_paths(name)
    print(f"Removing silver data and checkpoint for {name}: {S.path}, {S.checkpoint}")
    try:
        dbutils.fs.rm(S.path, recurse=True)
    except Exception as e:
        print(f"  (ignore) failed removing {S.path}: {e}")
    try:
        dbutils.fs.rm(S.checkpoint, recurse=True)
    except Exception as e:
        print(f"  (ignore) failed removing {S.checkpoint}: {e}")

# enriched target
S_enriched = silver_paths("encounters_enriched")
print(f"Removing enriched path and checkpoint: {S_enriched.path}, {S_enriched.checkpoint}")
try:
    dbutils.fs.rm(S_enriched.path, recurse=True)
except Exception as e:
    print(f"  (ignore) failed removing {S_enriched.path}: {e}")
try:
    dbutils.fs.rm(S_enriched.checkpoint, recurse=True)
except Exception as e:
    print(f"  (ignore) failed removing {S_enriched.checkpoint}: {e}")

print("Wipe complete.")